# Exploratory Data Analysis (EDA) - Sentiment Analysis

**Автор:** Новиков Максим Петрович  
**Группа:** БСБО-05-23

В этом ноутбуке проводится разведочный анализ данных для задачи анализа тональности текстов.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import string

# Настройки отображения
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

%matplotlib inline

## 1. Загрузка данных

In [ ]:
# Загрузка датасета
df = pd.read_csv('../data/sentiment_data.csv')

print(f"Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов")
print(f"\nСтолбцы: {list(df.columns)}")
print(f"\nТипы данных:\n{df.dtypes}")

In [ ]:
# Первые 10 строк
df.head(10)

## 2. Базовая статистика

In [ ]:
# Проверка на пропуски
print("Пропущенные значения:")
print(df.isnull().sum())

print(f"\nДубликаты: {df.duplicated().sum()}")

In [ ]:
# Распределение классов
label_counts = df['label'].value_counts()
print("Распределение классов:")
print(f"Положительные (1): {label_counts[1]} ({label_counts[1]/len(df)*100:.1f}%)")
print(f"Отрицательные (0): {label_counts[0]} ({label_counts[0]/len(df)*100:.1f}%)")

In [ ]:
# Визуализация распределения классов
fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#FF6B6B', '#4ECDC4']
bars = ax.bar(['Negative (0)', 'Positive (1)'], label_counts.values, color=colors)
ax.set_ylabel('Count')
ax.set_title('Distribution of Sentiment Classes')

# Добавляем числа на столбцы
for bar, count in zip(bars, label_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            str(count), ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.savefig('../artifacts/class_distribution.png', dpi=150)
plt.show()

## 3. Анализ текстов

In [ ]:
# Длина текстов
df['text_length'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

print("Статистика по длине текста (символы):")
print(df['text_length'].describe())

print("\nСтатистика по количеству слов:")
print(df['word_count'].describe())

In [ ]:
# Распределение длины текста по классам
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# По количеству символов
for label, color in zip([0, 1], colors):
    subset = df[df['label'] == label]['text_length']
    label_name = 'Positive' if label == 1 else 'Negative'
    axes[0].hist(subset, bins=20, alpha=0.7, label=label_name, color=color)
axes[0].set_xlabel('Text Length (characters)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Text Length by Sentiment')
axes[0].legend()

# По количеству слов
for label, color in zip([0, 1], colors):
    subset = df[df['label'] == label]['word_count']
    label_name = 'Positive' if label == 1 else 'Negative'
    axes[1].hist(subset, bins=15, alpha=0.7, label=label_name, color=color)
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of Word Count by Sentiment')
axes[1].legend()

plt.tight_layout()
plt.savefig('../artifacts/text_length_distribution.png', dpi=150)
plt.show()

In [ ]:
# Средняя длина текста по классам
print("Средняя длина текста по классам:")
print(df.groupby('label')[['text_length', 'word_count']].mean())

## 4. Анализ часто встречающихся слов

In [ ]:
def clean_text(text):
    """Очистка текста для анализа слов"""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def get_word_freq(texts):
    """Подсчёт частоты слов"""
    all_words = []
    for text in texts:
        words = clean_text(text).split()
        all_words.extend(words)
    return Counter(all_words)

# Частота слов для каждого класса
positive_texts = df[df['label'] == 1]['text']
negative_texts = df[df['label'] == 0]['text']

positive_freq = get_word_freq(positive_texts)
negative_freq = get_word_freq(negative_texts)

In [ ]:
# Топ-15 слов для каждого класса
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Положительные
top_positive = dict(positive_freq.most_common(15))
axes[0].barh(list(top_positive.keys())[::-1], list(top_positive.values())[::-1], color='#4ECDC4')
axes[0].set_xlabel('Frequency')
axes[0].set_title('Top 15 Words in Positive Reviews')

# Отрицательные
top_negative = dict(negative_freq.most_common(15))
axes[1].barh(list(top_negative.keys())[::-1], list(top_negative.values())[::-1], color='#FF6B6B')
axes[1].set_xlabel('Frequency')
axes[1].set_title('Top 15 Words in Negative Reviews')

plt.tight_layout()
plt.savefig('../artifacts/word_frequency.png', dpi=150)
plt.show()

## 5. Примеры текстов

In [ ]:
print("Примеры ПОЛОЖИТЕЛЬНЫХ отзывов:")
print("-" * 50)
for text in df[df['label'] == 1]['text'].sample(5, random_state=42).values:
    print(f"• {text}")

print("\n")
print("Примеры ОТРИЦАТЕЛЬНЫХ отзывов:")
print("-" * 50)
for text in df[df['label'] == 0]['text'].sample(5, random_state=42).values:
    print(f"• {text}")

## 6. Выводы по EDA

### Основные наблюдения:

1. **Размер данных:** Датасет содержит 100 отзывов - достаточно для демонстрации.

2. **Баланс классов:** Классы идеально сбалансированы (50/50) - не требуется применение техник балансировки.

3. **Качество данных:** Нет пропущенных значений и дубликатов.

4. **Длина текстов:** Средняя длина ~45-50 символов, ~8-9 слов. Тексты короткие и однородные.

5. **Ключевые слова:**
   - Положительные: "love", "great", "amazing", "excellent", "perfect"
   - Отрицательные: "terrible", "awful", "horrible", "worst", "disappointed"

### Рекомендации для моделирования:

1. Использовать TF-IDF для baseline модели (LogisticRegression)
2. Для улучшенной модели - DistilBERT как легковесный трансформер
3. Метрики: Accuracy, Precision, Recall, F1-score

In [ ]:
# Сохранение статистики
import os
os.makedirs('../artifacts', exist_ok=True)

stats = {
    'total_samples': len(df),
    'positive_samples': int(label_counts[1]),
    'negative_samples': int(label_counts[0]),
    'avg_text_length': float(df['text_length'].mean()),
    'avg_word_count': float(df['word_count'].mean())
}

import json
with open('../artifacts/eda_stats.json', 'w') as f:
    json.dump(stats, f, indent=2)

print("EDA статистика сохранена в artifacts/eda_stats.json")